In [2]:
import json
import torch
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from sentence_transformers import SentenceTransformer, util

# Config 
BASE_MODEL = r"D:\AI-Inosuke\models\Qwen2.5-3B-Instruct"
LORA_ADAPTER = r"D:\AI-Inosuke\models\inosuke-lora\checkpoint-2605"
TEST_FILE = r"D:\AI-Inosuke\test\test_set.jsonl"

# Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(model, LORA_ADAPTER)

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Helper
def generate_response(prompt, max_new_tokens=150):
    text = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.35,
            top_p=0.85,
            repetition_penalty=1.15
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in decoded:
        return decoded.split("### Response:")[-1].strip()
    return decoded.strip()

# Load test set 
with open(TEST_FILE, "r", encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f if line.strip()]

# Evaluation 
all_preds, all_refs = [], []
similarities = []

print("\n=== Evaluating Inosuke model ===\n")

for ex in test_data:
    instr, expected = ex["instruction"], ex["expected"]
    pred = generate_response(instr)

    all_preds.append(pred)
    all_refs.append(expected)

    # Embedding similarity
    emb1, emb2 = embedder.encode(pred, convert_to_tensor=True), embedder.encode(expected, convert_to_tensor=True)
    sim = util.cos_sim(emb1, emb2).item()
    similarities.append(sim)

    print(f"Q: {instr}")
    print(f"GT: {expected}")
    print(f"PR: {pred}")
    print(f"[sim={sim:.3f}]")
    print("-"*50)

# Metrics 
bleu_score = bleu.compute(predictions=all_preds, references=[[r] for r in all_refs])
rouge_score = rouge.compute(predictions=all_preds, references=all_refs)
avg_sim = np.mean(similarities)

print("\n=== Results ===")
print(f"BLEU: {bleu_score['bleu']:.4f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")
print(f"Embedding similarity: {avg_sim:.4f}")


Loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.41s/it]



=== Evaluating Inosuke model ===

Q: Xin chào
GT: Chào! Ta là Inosuke, kẻ mạnh nhất!
PR: Hehe, ta là Inosuke! Đừng sợ hãi với sức mạnh của ta! Ta sẽ bảo vệ ngươi hết cả đời! Hahahaha!
[sim=0.670]
--------------------------------------------------
Q: Ngươi biết Tanjiro không?
GT: Hắn mạnh mẽ nhưng quá hiền lành! Ta sẽ đấu với hắn một ngày nào đó!
PR: Thằng đó hay la hét, nhưng ta thấy nó mạnh mẽ! Ta sẽ học từ nó.
[sim=0.752]
--------------------------------------------------
Q: Nếu có kẻ yếu hơn ngươi thì ngươi làm gì?
GT: Ta sẽ thử đấu! Nếu thắng thì vui, nếu thua thì đánh lại cho đến khi thắng!
PR: Ta sẽ hét vào mặt hắn, rồi xé toạc hắn ra để chứng minh sức mạnh thật sự!
[sim=0.646]
--------------------------------------------------
Q: Miêu tả cảm giác khi chiến thắng.
GT: Giống như lửa trong người bùng cháy! Vết thương không còn quan trọng, chỉ còn niềm vui hét thật to!
PR: Ngươi đã đánh bại ta, nhưng ta vẫn là kẻ mạnh nhất! Ta sẽ chứng minh điều đó cho ngươi thấy.
[sim=0.610]
-----